In [17]:
%cd /content/drive/MyDrive/DataAnalysis/書籍/study_BERT/mycode

/content/drive/MyDrive/DataAnalysis/書籍/study_BERT/mycode


In [5]:
!mkdir ch7

In [18]:
%cd ./ch7

/content/drive/MyDrive/DataAnalysis/書籍/study_BERT/mycode/ch7


In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.10

     |████████████████████████████████| 2.1 MB 14.2 MB/s 
     |████████████████████████████████| 486 kB 43.8 MB/s 
     |████████████████████████████████| 13.4 MB 40.1 MB/s 
     |████████████████████████████████| 841 kB 37.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 136 kB 53.4 MB/s 
     |████████████████████████████████| 829 kB 43.0 MB/s 
     |████████████████████████████████| 176 kB 44.9 MB/s 
     |████████████████████████████████| 596 kB 44.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.9 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 51.5 MB/s 
     |████████████████████████████████| 94 kB 659 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=5a04087de41acaadb5c881d8c52ed02bb3e676a9af9090843d7345a951cfb97e
  Stored in directory: /root/.cache/

In [2]:
import random
import glob
from tqdm import tqdm
import json

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# 事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [8]:
# マルチラベル分類
class BertForSequenceClassificationMultiLabel(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()
        # モデルのロード
        self.bert = BertModel.from_pretrained(model_name)
        # 線形変換
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask = None, token_type_ids = None, labels = None):
        bert_output = self.bert(input_ids = input_ids, attention_mask = attention_mask,
                                token_type_ids = token_type_ids)
        last_hidden_state = bert_output.last_hidden_state

        # [PAD]以外のトークンで平均
        # batch, sequence, hidden_size →batch, hidden_size
        averaged_hidden_state = (last_hidden_state * attention_mask.unsqueeze(-1)).sum(1)/ attention_mask.sum(1, keepdim = True)

        # 線形変換
        # batch, num_labels
        scores = self.linear(averaged_hidden_state)

        # 出力の形式整理
        output = {'logits':scores}

        # 損失計算
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output['loss'] = loss
        
        # 属性でアクセスできるように
        output = type('bert_output', (object,), output)

        return output


In [9]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_scml = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=2
) 
bert_scml = bert_scml.cuda()

In [5]:
print(bert_scml)

BertForSequenceClassificationMultiLabel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [10]:
# 試してみる
text_list = ['今日の仕事はうまくいったが、体調があまり良くない。',
             '昨日は楽しかった。']

labels_list = [[1, 1], [0, 1]]

# データの符号化→GPU
encoding = tokenizer(text_list, padding = 'longest', return_tensors = 'pt')
encoding = {k:v.cuda() for k, v in encoding.items()}
labels = torch.tensor(labels_list).cuda()

with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits

# スコアが正ならカテゴリ選択
labels_predicted = (scores > 0).int()

# 精度の計算
num_correct = (labels_predicted == labels).all(-1).sum().item()
accuracy = num_correct/labels.size(0)

In [14]:
labels_predicted

tensor([[0, 0],
        [0, 0]], device='cuda:0', dtype=torch.int32)

In [15]:
encoding = tokenizer(
    text_list, 
    padding='longest',  
    return_tensors='pt'
)
encoding['labels'] = torch.tensor(labels_list) # 入力にlabelsを含める。
encoding = { k: v.cuda() for k, v in encoding.items() }

output = bert_scml(**encoding)
loss = output.loss 

# 実際にやってみる

In [19]:
# データのダウンロード
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
# データの解凍
!unzip chABSA-dataset.zip 

--2022-04-10 05:08:55--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.136.62
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.136.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K   625KB/s    in 1.1s    

2022-04-10 05:08:57 (625 KB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

Archive:  chABSA-dataset.zip
   creating: chABSA-dataset/
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflating:

In [20]:
# 1つ読み込んでみる
data = json.load(open('chABSA-dataset/e00030_ann.json'))
print(data['sentences'][0])

{'sentence_id': 0, 'sentence': '当期におけるわが国経済は、景気は緩やかな回復基調が続き、設備投資の持ち直し等を背景に企業収益は改善しているものの、海外では、資源国等を中心に不透明な状況が続き、為替が急激に変動するなど、依然として先行きが見通せない状況で推移した', 'opinions': [{'target': 'わが国経済', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 6, 'to': 11}, {'target': '景気', 'category': 'NULL#general', 'polarity': 'positive', 'from': 13, 'to': 15}, {'target': '設備投資', 'category': 'NULL#general', 'polarity': 'positive', 'from': 28, 'to': 32}, {'target': '企業収益', 'category': 'NULL#general', 'polarity': 'positive', 'from': 42, 'to': 46}, {'target': '資源国等', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 62, 'to': 66}, {'target': '為替', 'category': 'NULL#general', 'polarity': 'negative', 'from': 80, 'to': 82}]}


In [21]:
category_id = {'negative':0, 'neutral':1, 'positive':2}

dataset = []
for file in glob.glob('chABSA-dataset/*.json'):
    data = json.load(open(file))
    # ラベルと文章を整形
    for sentence in data['sentences']:
        # テキスト格納
        text = sentence['sentence']
        # ラベル格納
        labels = [0, 0, 0]
        for opinion in sentence['opinions']:
            labels[category_id[opinion['polarity']]] = 1
        sample = {'text': text, 'labels': labels}
        dataset.append(sample)

In [22]:
print(dataset[0])

{'text': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております', 'labels': [0, 1, 1]}


In [31]:
# データセットの分割

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

max_length = 128
dataset_for_loader = []
for sample in dataset:
    text = sample['text']
    labels = sample['labels']
    encoding = tokenizer(text, max_length = max_length, padding = 'max_length', truncation = True)
    encoding['labels'] = labels
    encoding = {k: torch.tensor(v) for k, v in encoding.items()}
    dataset_for_loader.append(encoding)

# データセットの分割
random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

# データローダー
dataloader_train = DataLoader(dataset_train, batch_size = 32, shuffle = True)
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [28]:
dataset_for_loader[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    2, 10025,  2943,   774,     9,     6, 23186,  1275,     5,  4331,
            12,    31,  4245, 24121,    96,  7273,    68, 19548,  6471,  1275,
             5, 13440,  1179,   774,    11, 20057,    15,    10,    45,    64,
           225,     6, 13791,     9,  2157,    15,    10,  1036,     6,  1926,
          3127,   774,    49, 12599,   774,   320, 13791,    14,  2643,    15,
            10,    45,    64,    40,     6,  1895,  1562,    12,     9, 13791,
             9,  2643,  6785, 28454,  3913,    10,     

In [32]:
# 7-13
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters() 
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        ) 

    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)
        scores = output.logits
        labels_predicted = ( scores > 0 ).int()
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct/scores.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

trainer = pl.Trainer(
    gpus=1, 
    max_epochs=5,
    callbacks = [checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, 
    num_labels=3, 
    lr=1e-5
)
trainer.fit(model, dataloader_train, dataloader_val)
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory model/ exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                                    | Params
----------------------------------------------------------------------
0 | bert_scml | BertForSequenceClassificationMultiLabel | 110 M 
----------------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.479   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.8897958993911743}
--------------------------------------------------------------------------------
Accuracy: 0.89


In [33]:
# 入力する文章
text_list = [
    "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "昨年から黒字が減少した。",
    "今日の飲み会は楽しかった。"
]

# モデルのロード
best_model_path = checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml.cuda()

# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() }

# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')

--
入力：今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。
出力：[1, 0, 1]
--
入力：昨年から黒字が減少した。
出力：[1, 0, 0]
--
入力：今日の飲み会は楽しかった。
出力：[0, 0, 0]
